In [119]:
path = 'Creative-Arcades-6296.pdf'
fileobj = open(path,'rb')
#install camelot
import camelot
import pandas as pd


To correctly extract values of different variables in the pdf file, we need to use the argument called **columns** to specify which is the seperator line of each column. By experimentation, the best value for this argument ís **columns=['100,500']**.

However, this argument cannot be used with pages having more than one table. Therefore, we temporarily do not consider pages having more than one table. 

Room for improvement: Maybe there is some ways to specify columns in each table, so we can scan all pages at the same time without considering which pages have more than one table.

In [120]:
#this code is to return a string that specify which pages are chosen to extract, only including pages having one table.
string = '1'
for i in range(137):
    tabs = camelot.read_pdf(path, pages=str(i),flavor='stream')
    if len(list(tabs)) > 1:
        new = '-' + str(i-1) + ', ' + str(i+1)
        string += new
string += '-end'

Fortunately, there are not too many pages having more than one table. So we still can automatically extract many data.

In [121]:
tables = camelot.read_pdf(path, pages=string, flavor='stream', columns=['100,150'], split_text=True)

Up to this point, we modify the data. Because there is some game title not having any information about type, camilot extracts some table with two columns (without type) and some with three columns (with type). To correctly concenate all tables, we need to specify the structure of the table and modify the data to this structure.

In [122]:
#identify the structure of the dataset
dict = { 'Number': { }, 
        'Title': { },
        'Type': { },}
new_df = pd.DataFrame(dict)

In [123]:
#the function to modify each table
def modify_table(t):
    if len(list(t.columns)) == 3:
        t.rename(columns={0: 'Number', 1: 'Type', 2: 'Title'}, inplace=True)
        t = t[['Number', 'Title', 'Type']]
                        
    if len(list(t.columns)) == 2: 
        t.rename(columns={0: 'Number', 1: 'Title'}, inplace=True)   
    
    return t

In [124]:
#Apply the modifying function to each table and concatnate every table.
for i in range(len(tables)):
        temp_df = tables[i].df
        modify_table(temp_df)
        new_df = pd.concat([new_df, temp_df], ignore_index=True)                

In [125]:
new_df

,Number,Title,Type
0,NO.,English,TYPE
1,1,Tekken,3D
2,2,Tekken 2,3D
3,3,Tekken 3,3D
4,4,Mortal Kombat,3D
...,...,...,...
6154,6292,World Beach Volley,
6155,6293,W-Ring - The Double Rings,
6156,6294,Xevious - Fardraut Saga,
6157,6295,Youkai Douchuuki,


At this point, we have extracted 97.84% of the records in the pdf file. However, I want to reach the number of 100%, so I still keep going. From the string variable, we know that pages having more than one table is pages 3, 7 and 22.


In coding, I always want to try to make the code reproducible. But in this case, there is only one table in each page and camelot extracts wrongly for some reasons. Because the failure of each page is different and I also do not quite understand the mechanism of Camelot, so I try to handle these pages each by each. 

Note: This way is only applicable when the number of pages is small (in this case, only 3 pages). I still need to figure out the way to address this in a more systematic way.

In [126]:
table = camelot.read_pdf(path, pages='3',flavor='stream')
temp_df = table[1].df
modify_table(temp_df)
new_df = pd.concat([new_df, temp_df], ignore_index=True)      

In [127]:
table = camelot.read_pdf(path, pages='7',flavor='stream',split_text=True)
temp_df = table[1].df
temp_df.loc[temp_df[2].str.contains('1942'),1] = '1942'
temp_df[2] = ""
temp_df.loc[temp_df[1].str.contains('TRACKBALL'),2] = 'TRACKBAll'  
temp_df.loc[temp_df[1].str.contains('3/4P'),2] = '3/4P'
temp_df[1] = temp_df[1].str.replace('TRACKBALL','')
temp_df[1]  = temp_df[1].str.replace('3/4P\n','')
temp_df[1]  = temp_df[1].str.replace('\n3/4P','')
temp_df.rename(columns={0: 'Number', 1: 'Title', 2: 'Type'}, inplace=True)
new_df = pd.concat([new_df, temp_df], ignore_index=True)    

In [128]:
table = camelot.read_pdf(path, pages='22',flavor='stream',split_text=True)
temp_df = table[0].df
temp_df.loc[temp_df[1].str.contains('TRACKBALL'),2] = 'TRACKBAll'  
temp_df[1] = temp_df[1].str.replace('TRACKBALL','')
temp_df.rename(columns={0: 'Number', 1: 'Title', 2: 'Type'}, inplace=True)
new_df = pd.concat([new_df, temp_df], ignore_index=True)  

In [129]:
new_df.drop(index=new_df.index[0], 
        axis=0, 
        inplace=True)

In [130]:
new_df['Number'] = pd.to_numeric(new_df['Number'])
new_df = new_df.sort_values('Number')
new_df.set_index('Number', inplace=True)
new_df.to_csv('new_df_final.csv')